In [1]:
import json
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [2]:
train = json.load(open('../_data/train.json'))
test = json.load(open('../_data/test.json'))

In [3]:
train_as_text = [' '.join(sample['ingredients']).lower() for sample in train]
train_cuisine = [sample['cuisine'] for sample in train]

test_as_text = [' '.join(sample['ingredients']).lower() for sample in test]

In [4]:
train_as_text[283]

'bertolli® classico olive oil boneless skinless chicken breast halves eggs linguine chicken broth bacon, crisp-cooked and crumbled bertolli vineyard premium collect marinara with burgundi wine sauc bread crumb fresh shredded mozzarella cheese'

In [5]:
from time import strftime

def print_time():
    print(strftime('%y%m%d-%H%M%S'))
    return

In [6]:
from sklearn.pipeline import FeatureUnion

In [7]:
def itself(x):
    return x

In [8]:
import re

In [9]:
SPEC_REMOVE = re.compile(r'(\'|\’|\(.*oz.*\)|(\()|(\)))')
SPEC_AND = re.compile(r'\&')
SPEC_ELSE = re.compile(r'[^\w\s\%_]')

def clean_ingr(ingr):
    ingr = re.sub(SPEC_REMOVE, '', ingr)
    ingr = re.sub(SPEC_AND, 'and', ingr)
    ingr = re.sub(SPEC_ELSE, ' ', ingr)
    return ' '.join(ingr.split())

In [10]:
def get_ingrs(given):
    ingrs = [[clean_ingr(i).lower() for i in recipe['ingredients']] for recipe in given]
    return ingrs

def get_labels(given):
    return [r['cuisine'] for r in given]

In [11]:
%%time
print_time()
train_ingrs = get_ingrs(train)
print_time()

180410-160718
180410-160720
CPU times: user 1.94 s, sys: 11.6 ms, total: 1.95 s
Wall time: 1.95 s


In [12]:
def combine_words(ilist):
    return ' '.join(ilist)

In [13]:
%%time
dvec_all = FeatureUnion([
        ("ingrs", TfidfVectorizer(strip_accents='unicode',
                                  tokenizer=itself,
                                  preprocessor=itself)),
        ("words", TfidfVectorizer(strip_accents='unicode',
                                  preprocessor=combine_words,
                                  #stop_words='english',
                                  )),
        ]).fit(get_ingrs(train+test))

CPU times: user 3.26 s, sys: 20.1 ms, total: 3.28 s
Wall time: 3.27 s


In [14]:
%%time
train_mat = dvec_all.transform(train_ingrs)
train_mat

CPU times: user 1.01 s, sys: 15.9 ms, total: 1.03 s
Wall time: 1.02 s


In [15]:
import xgboost as xgb
from xgboost import XGBClassifier

In [16]:
train_mat.shape

(39774, 10228)

In [17]:
feats = np.array(dvec_all.get_feature_names())

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [19]:
train_labels0 = [x['cuisine'] for x in train]
train_labels = le.fit_transform(train_labels0)

In [20]:
le.classes_

array(['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino',
       'french', 'greek', 'indian', 'irish', 'italian', 'jamaican',
       'japanese', 'korean', 'mexican', 'moroccan', 'russian',
       'southern_us', 'spanish', 'thai', 'vietnamese'],
      dtype='<U12')

In [21]:
train_labels

array([ 6, 16,  4, ...,  8,  3, 13])

In [35]:
#Choose all predictors except target & IDcols
#predictors = [x for x in train.columns if x not in [target, IDcol]]

xgb1 = XGBClassifier(
    learning_rate=0.05,
    n_estimators=1000,
    max_depth=7,
    min_child_weight=1,
    gamma=0.35,
    subsample=1.0,
    colsample_bytree=0.5,
    reg_alpha=0.001,
    reg_lambda=1.0,
    objective='multi:softmax',
    scale_pos_weight=1,
    num_class=20,
    n_jobs=8,
    seed=27)

In [36]:
xgb_param = xgb1.get_xgb_params()

In [37]:
xgtrain = xgb.DMatrix(data=train_mat,
                      label=train_labels,
                      feature_names=feats)

In [38]:
%%time
print_time()

cvresult = xgb.cv(xgb_param,
                  xgtrain,
                  num_boost_round=xgb1.get_params()['n_estimators'],
                  nfold=3,
                  metrics='merror',
                  early_stopping_rounds=50,
                  verbose_eval=True)

print_time()

180410-164859
[0]	train-merror:0.369852+0.0114525	test-merror:0.402801+0.00600908
[1]	train-merror:0.319254+0.00902216	test-merror:0.357922+0.0075838
[2]	train-merror:0.298122+0.007432	test-merror:0.340524+0.00736964
[3]	train-merror:0.285564+0.00819942	test-merror:0.329889+0.00477457
[4]	train-merror:0.276978+0.00585182	test-merror:0.323+0.00577866
[5]	train-merror:0.270025+0.00450574	test-merror:0.315407+0.0056136
[6]	train-merror:0.264595+0.00437526	test-merror:0.312365+0.00585104
[7]	train-merror:0.259805+0.00430871	test-merror:0.308065+0.00577882
[8]	train-merror:0.256436+0.00512424	test-merror:0.306633+0.00527425
[9]	train-merror:0.251785+0.00510781	test-merror:0.303364+0.00476195
[10]	train-merror:0.249158+0.00447114	test-merror:0.301805+0.00424176
[11]	train-merror:0.246204+0.00396812	test-merror:0.300146+0.00393193
[12]	train-merror:0.243564+0.00424736	test-merror:0.298612+0.00421917
[13]	train-merror:0.241075+0.00411734	test-merror:0.297732+0.00381329
[14]	train-merror:0.2390

[118]	train-merror:0.12913+0.00279327	test-merror:0.245085+0.00331455
[119]	train-merror:0.128551+0.00292326	test-merror:0.244909+0.00296443
[120]	train-merror:0.128061+0.0031149	test-merror:0.244657+0.00299336
[121]	train-merror:0.127332+0.00297846	test-merror:0.24423+0.00294317
[122]	train-merror:0.126603+0.0027903	test-merror:0.244356+0.00328025
[123]	train-merror:0.125924+0.00284034	test-merror:0.24428+0.00307769
[124]	train-merror:0.125434+0.00275868	test-merror:0.243853+0.00314691
[125]	train-merror:0.124818+0.00278472	test-merror:0.243828+0.00342365
[126]	train-merror:0.124139+0.0025987	test-merror:0.243375+0.00324076
[127]	train-merror:0.123561+0.0025515	test-merror:0.243199+0.00343473
[128]	train-merror:0.123133+0.00242372	test-merror:0.243023+0.00347687
[129]	train-merror:0.122731+0.00230719	test-merror:0.242696+0.0032807
[130]	train-merror:0.121977+0.00248058	test-merror:0.24247+0.00352283
[131]	train-merror:0.121499+0.00252043	test-merror:0.242219+0.00339351
[132]	train-mer

[234]	train-merror:0.083723+0.00142734	test-merror:0.227334+0.00361137
[235]	train-merror:0.0833963+0.00143058	test-merror:0.227259+0.00357341
[236]	train-merror:0.0831197+0.00136707	test-merror:0.227184+0.00337135
[237]	train-merror:0.08278+0.00139891	test-merror:0.227133+0.00333133
[238]	train-merror:0.082642+0.00144113	test-merror:0.227158+0.00307683
[239]	train-merror:0.0823903+0.00134688	test-merror:0.226983+0.0032356
[240]	train-merror:0.0820387+0.00140991	test-merror:0.226781+0.00321554
[241]	train-merror:0.0817367+0.00128655	test-merror:0.226555+0.00326431
[242]	train-merror:0.0816113+0.00134613	test-merror:0.22648+0.00343609
[243]	train-merror:0.081196+0.00142362	test-merror:0.22663+0.00354142
[244]	train-merror:0.0809323+0.00140306	test-merror:0.22653+0.00357622
[245]	train-merror:0.0808693+0.00130711	test-merror:0.226479+0.00358955
[246]	train-merror:0.0806053+0.00131112	test-merror:0.22653+0.00379655
[247]	train-merror:0.0803287+0.00145173	test-merror:0.226304+0.00377224
[2

[350]	train-merror:0.0590963+0.00214397	test-merror:0.220647+0.00407234
[351]	train-merror:0.0589957+0.00213113	test-merror:0.220571+0.00391124
[352]	train-merror:0.0588323+0.00224325	test-merror:0.220621+0.00389369
[353]	train-merror:0.058644+0.00229831	test-merror:0.220672+0.00403557
[354]	train-merror:0.0584807+0.00228321	test-merror:0.220672+0.00398267
[355]	train-merror:0.0583297+0.00225309	test-merror:0.220672+0.00403652
[356]	train-merror:0.0581787+0.00228898	test-merror:0.220546+0.0039466
[357]	train-merror:0.0580153+0.00229317	test-merror:0.22032+0.00384079
[358]	train-merror:0.0578897+0.00230079	test-merror:0.220219+0.00376962
[359]	train-merror:0.057751+0.00215892	test-merror:0.220244+0.00384032
[360]	train-merror:0.057563+0.00224622	test-merror:0.22032+0.00394732
[361]	train-merror:0.0573993+0.0022818	test-merror:0.220219+0.00398314
[362]	train-merror:0.0573113+0.00224342	test-merror:0.220345+0.00396559
[363]	train-merror:0.057261+0.00227474	test-merror:0.220345+0.00391294


[466]	train-merror:0.0426157+0.00106699	test-merror:0.217378+0.00354628
[467]	train-merror:0.042515+0.00105795	test-merror:0.217127+0.00337862
[468]	train-merror:0.0424147+0.000996313	test-merror:0.217202+0.00343563
[469]	train-merror:0.0422767+0.000992519	test-merror:0.217076+0.00332724
[470]	train-merror:0.0422133+0.000982037	test-merror:0.217152+0.00344056
[471]	train-merror:0.0421383+0.000956912	test-merror:0.217102+0.00339799
[472]	train-merror:0.0420503+0.000934285	test-merror:0.217102+0.00340502
[473]	train-merror:0.0419247+0.000972039	test-merror:0.217102+0.00325082
[474]	train-merror:0.0419117+0.00100539	test-merror:0.217152+0.00323784
[475]	train-merror:0.0418237+0.00101352	test-merror:0.217127+0.00328287
[476]	train-merror:0.04161+0.000924696	test-merror:0.217127+0.00316751
[477]	train-merror:0.0414843+0.000853806	test-merror:0.217026+0.00313689
[478]	train-merror:0.041384+0.000871708	test-merror:0.217051+0.00309405
[479]	train-merror:0.041233+0.000851137	test-merror:0.21707

[580]	train-merror:0.0315657+0.000585321	test-merror:0.214085+0.00380471
[581]	train-merror:0.0314653+0.000631885	test-merror:0.213984+0.00378668
[582]	train-merror:0.031327+0.000529925	test-merror:0.213984+0.00384079
[583]	train-merror:0.0312393+0.000551125	test-merror:0.214034+0.00376962
[584]	train-merror:0.031126+0.000602037	test-merror:0.214009+0.00380497
[585]	train-merror:0.030975+0.000692036	test-merror:0.213934+0.0038052
[586]	train-merror:0.0308747+0.00070908	test-merror:0.213883+0.00376962
[587]	train-merror:0.0307737+0.000717446	test-merror:0.213808+0.00371597
[588]	train-merror:0.0306357+0.000713453	test-merror:0.213859+0.00369845
[589]	train-merror:0.0306353+0.000692484	test-merror:0.213884+0.00371676
[590]	train-merror:0.0305853+0.000695936	test-merror:0.213883+0.00366442
[591]	train-merror:0.030422+0.000689024	test-merror:0.213959+0.00371676
[592]	train-merror:0.030359+0.000716313	test-merror:0.213984+0.00368324
[593]	train-merror:0.0302583+0.00077504	test-merror:0.2140

[694]	train-merror:0.0233447+0.000785513	test-merror:0.212501+0.00391922
[695]	train-merror:0.0232817+0.000785521	test-merror:0.212501+0.00386769
[696]	train-merror:0.0231937+0.000800575	test-merror:0.212475+0.00385756
[697]	train-merror:0.0231437+0.000801366	test-merror:0.212526+0.00393889
[698]	train-merror:0.0231057+0.000801366	test-merror:0.212475+0.00390342
[699]	train-merror:0.02303+0.000787931	test-merror:0.212551+0.00385803
[700]	train-merror:0.0230177+0.000803712	test-merror:0.212476+0.0037414
[701]	train-merror:0.022967+0.000819991	test-merror:0.212425+0.00370591
[702]	train-merror:0.022917+0.000793477	test-merror:0.212425+0.0037023
[703]	train-merror:0.0228543+0.000796718	test-merror:0.212324+0.00369044
[704]	train-merror:0.0227787+0.000746637	test-merror:0.212199+0.00359982
[705]	train-merror:0.0227533+0.000739424	test-merror:0.212148+0.00367526
[706]	train-merror:0.022703+0.000716313	test-merror:0.212123+0.00366034
[707]	train-merror:0.022653+0.000702319	test-merror:0.2120

[808]	train-merror:0.017373+0.000497803	test-merror:0.211193+0.00407018
[809]	train-merror:0.0173227+0.000489315	test-merror:0.211168+0.00406179
[810]	train-merror:0.0172977+0.00047166	test-merror:0.211118+0.00414009
[811]	train-merror:0.0172723+0.000534272	test-merror:0.211143+0.00410502
[812]	train-merror:0.0171847+0.000498509	test-merror:0.211243+0.00411831
[813]	train-merror:0.017134+0.000544309	test-merror:0.211269+0.00409134
[814]	train-merror:0.0170083+0.000508621	test-merror:0.211168+0.00413534
[815]	train-merror:0.0169833+0.000489093	test-merror:0.211168+0.00413534
[816]	train-merror:0.016996+0.000498769	test-merror:0.211168+0.00409715
[817]	train-merror:0.0169203+0.000477445	test-merror:0.211143+0.00413244
[818]	train-merror:0.0168703+0.000494835	test-merror:0.211118+0.00415748
[819]	train-merror:0.0167697+0.00047166	test-merror:0.211017+0.00406949
[820]	train-merror:0.0167573+0.000489093	test-merror:0.211143+0.00421024
[821]	train-merror:0.0166943+0.000473422	test-merror:0.2

[922]	train-merror:0.0130613+0.000625535	test-merror:0.210766+0.00417908
[923]	train-merror:0.013011+0.000617898	test-merror:0.210715+0.0041435
[924]	train-merror:0.0129733+0.000622695	test-merror:0.21069+0.00412892
[925]	train-merror:0.0129733+0.000628809	test-merror:0.210691+0.00418362
[926]	train-merror:0.0129607+0.000653447	test-merror:0.21064+0.0042547
[927]	train-merror:0.0128853+0.000698562	test-merror:0.21059+0.00432118
[928]	train-merror:0.0128727+0.000679144	test-merror:0.210539+0.00428559
[929]	train-merror:0.0127973+0.000641234	test-merror:0.21059+0.00416172
[930]	train-merror:0.012772+0.000632229	test-merror:0.21059+0.0042738
[931]	train-merror:0.0127093+0.000585321	test-merror:0.21059+0.00428241
[932]	train-merror:0.0126967+0.000581499	test-merror:0.210716+0.00431351
[933]	train-merror:0.012659+0.000602037	test-merror:0.21069+0.00429353
[934]	train-merror:0.012596+0.000594107	test-merror:0.21069+0.00424204
[935]	train-merror:0.012533+0.00055973	test-merror:0.210539+0.0041

In [39]:
cvresult

,test-merror-mean,test-merror-std,train-merror-mean,train-merror-std
0,0.402801,0.006009,0.369852,0.011453
1,0.357922,0.007584,0.319254,0.009022
2,0.340524,0.007370,0.298122,0.007432
3,0.329889,0.004775,0.285564,0.008199
4,0.323000,0.005779,0.276978,0.005852
5,0.315407,0.005614,0.270025,0.004506
6,0.312365,0.005851,0.264595,0.004375
7,0.308065,0.005779,0.259805,0.004309
8,0.306633,0.005274,0.256436,0.005124
9,0.303364,0.004762,0.251785,0.005108


In [40]:
cvresult.shape[0]

888

LR 0.1

In [28]:
#Choose all predictors except target & IDcols
#predictors = [x for x in train.columns if x not in [target, IDcol]]

xgb1 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=7,
    min_child_weight=1,
    gamma=0.35,
    subsample=1.0,
    colsample_bytree=0.5,
    reg_alpha=0.001,
    reg_lambda=1.0,
    objective='multi:softmax',
    scale_pos_weight=1,
    num_class=20,
    n_jobs=8,
    seed=27)

In [29]:
xgb_param = xgb1.get_xgb_params()

In [30]:
xgtrain = xgb.DMatrix(data=train_mat,
                      label=train_labels,
                      feature_names=feats)

In [31]:
%%time
print_time()

cvresult = xgb.cv(xgb_param,
                  xgtrain,
                  num_boost_round=xgb1.get_params()['n_estimators'],
                  nfold=3,
                  metrics='merror',
                  early_stopping_rounds=50,
                  verbose_eval=True)

print_time()

180410-163117
[0]	train-merror:0.369852+0.0114525	test-merror:0.402801+0.00600908
[1]	train-merror:0.31239+0.00532797	test-merror:0.352869+0.006116
[2]	train-merror:0.286707+0.00373857	test-merror:0.332328+0.0082481
[3]	train-merror:0.274953+0.00430963	test-merror:0.323427+0.0086399
[4]	train-merror:0.265362+0.00355126	test-merror:0.317619+0.00700308
[5]	train-merror:0.256927+0.00252331	test-merror:0.312591+0.0064973
[6]	train-merror:0.250402+0.00317247	test-merror:0.308141+0.00585316
[7]	train-merror:0.245336+0.00289715	test-merror:0.304495+0.00638673
[8]	train-merror:0.241188+0.00301867	test-merror:0.301001+0.00542848
[9]	train-merror:0.236449+0.00277506	test-merror:0.298512+0.00551974
[10]	train-merror:0.231973+0.00317524	test-merror:0.296048+0.00555877
[11]	train-merror:0.228629+0.00310933	test-merror:0.294061+0.00578467
[12]	train-merror:0.22472+0.00325033	test-merror:0.290868+0.00560803
[13]	train-merror:0.221564+0.00281211	test-merror:0.288907+0.00545125
[14]	train-merror:0.2181

[118]	train-merror:0.0824913+0.00163142	test-merror:0.229597+0.00357338
[119]	train-merror:0.082026+0.00177412	test-merror:0.229597+0.00357974
[120]	train-merror:0.0814227+0.00184127	test-merror:0.22927+0.00376888
[121]	train-merror:0.0806053+0.00186405	test-merror:0.228918+0.00378792
[122]	train-merror:0.0800647+0.00193564	test-merror:0.228868+0.00332767
[123]	train-merror:0.0794613+0.00196556	test-merror:0.228617+0.00346685
[124]	train-merror:0.079097+0.00206695	test-merror:0.228466+0.0034734
[125]	train-merror:0.078632+0.00218951	test-merror:0.228214+0.00339703
[126]	train-merror:0.078041+0.00200285	test-merror:0.228063+0.00377112
[127]	train-merror:0.0775883+0.00182318	test-merror:0.227862+0.00352507
[128]	train-merror:0.0770227+0.00178645	test-merror:0.227711+0.00375786
[129]	train-merror:0.0766203+0.00170695	test-merror:0.227737+0.00371076
[130]	train-merror:0.076042+0.00183373	test-merror:0.227812+0.00398492
[131]	train-merror:0.0754893+0.00177704	test-merror:0.227611+0.0039943


[233]	train-merror:0.042264+0.0012257	test-merror:0.21866+0.00416745
[234]	train-merror:0.0419747+0.00120553	test-merror:0.218635+0.00431811
[235]	train-merror:0.0417487+0.00114928	test-merror:0.21866+0.00417655
[236]	train-merror:0.0415597+0.00123966	test-merror:0.21861+0.00409066
[237]	train-merror:0.0413587+0.00123435	test-merror:0.218384+0.00414107
[238]	train-merror:0.04112+0.00117102	test-merror:0.21851+0.00432907
[239]	train-merror:0.0408937+0.00119318	test-merror:0.218535+0.00450283
[240]	train-merror:0.0407427+0.00110545	test-merror:0.218509+0.00427401
[241]	train-merror:0.0405163+0.00111206	test-merror:0.21851+0.00448168
[242]	train-merror:0.0402273+0.00112302	test-merror:0.21861+0.00423535
[243]	train-merror:0.039938+0.00112417	test-merror:0.218384+0.00439359
[244]	train-merror:0.0396743+0.0011991	test-merror:0.218384+0.00445531
[245]	train-merror:0.0395233+0.00110211	test-merror:0.218333+0.00441594
[246]	train-merror:0.039272+0.000944401	test-merror:0.218359+0.004445
[247]	

[348]	train-merror:0.022716+0.00079714	test-merror:0.214789+0.0043655
[349]	train-merror:0.0227033+0.000809418	test-merror:0.214764+0.00440035
[350]	train-merror:0.02259+0.000777373	test-merror:0.214713+0.00429593
[351]	train-merror:0.0225397+0.000750849	test-merror:0.214688+0.00444854
[352]	train-merror:0.022452+0.00079714	test-merror:0.214562+0.0045316
[353]	train-merror:0.022263+0.000841285	test-merror:0.214537+0.00462546
[354]	train-merror:0.0221503+0.000820115	test-merror:0.214512+0.00460296
[355]	train-merror:0.022087+0.000833403	test-merror:0.214361+0.00448303
[356]	train-merror:0.0219113+0.000907513	test-merror:0.214286+0.00442446
[357]	train-merror:0.0218357+0.000907069	test-merror:0.214235+0.00443492
[358]	train-merror:0.0216977+0.000910755	test-merror:0.214286+0.00448289
[359]	train-merror:0.0215717+0.000927384	test-merror:0.214311+0.00445813
[360]	train-merror:0.021484+0.000934735	test-merror:0.214361+0.00443567
[361]	train-merror:0.0213457+0.000961231	test-merror:0.214386+

[462]	train-merror:0.0127973+0.000251012	test-merror:0.212727+0.00425969
[463]	train-merror:0.0127593+0.000204891	test-merror:0.212626+0.00433844
[464]	train-merror:0.0127343+0.000205177	test-merror:0.212652+0.00437315
[465]	train-merror:0.0126587+0.000230988	test-merror:0.212903+0.00433761
[466]	train-merror:0.0126337+0.000293624	test-merror:0.212953+0.00439019
[467]	train-merror:0.0125207+0.000240655	test-merror:0.212903+0.00442331
[468]	train-merror:0.0124577+0.000195633	test-merror:0.213028+0.00450541
[469]	train-merror:0.01242+0.000195161	test-merror:0.212979+0.00441276
[470]	train-merror:0.012395+0.000180106	test-merror:0.212852+0.00436808
[471]	train-merror:0.0123197+0.000230988	test-merror:0.213004+0.00445779
[472]	train-merror:0.0122443+0.000258065	test-merror:0.212802+0.00439019
[473]	train-merror:0.0122317+0.00027595	test-merror:0.212953+0.00433312
[474]	train-merror:0.012156+0.000279671	test-merror:0.212878+0.00423761
[475]	train-merror:0.0121313+0.000291095	test-merror:0.2

In [33]:
cvresult

,test-merror-mean,test-merror-std,train-merror-mean,train-merror-std
0,0.402801,0.006009,0.369852,0.011453
1,0.352869,0.006116,0.312390,0.005328
2,0.332328,0.008248,0.286707,0.003739
3,0.323427,0.008640,0.274953,0.004310
4,0.317619,0.007003,0.265362,0.003551
5,0.312591,0.006497,0.256927,0.002523
6,0.308141,0.005853,0.250402,0.003172
7,0.304495,0.006387,0.245336,0.002897
8,0.301001,0.005428,0.241188,0.003019
9,0.298512,0.005520,0.236449,0.002775


In [34]:
cvresult.shape[0]

464

LR 0.3

In [25]:
%%time
print_time()

cvresult = xgb.cv(xgb_param,
                  xgtrain,
                  num_boost_round=xgb1.get_params()['n_estimators'],
                  nfold=3,
                  metrics='merror',
                  early_stopping_rounds=50,
                  verbose_eval=True)

print_time()

180410-161713
[0]	train-merror:0.369852+0.0114525	test-merror:0.402801+0.00600908
[1]	train-merror:0.300787+0.00687841	test-merror:0.348469+0.00715111
[2]	train-merror:0.271848+0.00607218	test-merror:0.328104+0.00368015
[3]	train-merror:0.254362+0.00566253	test-merror:0.317846+0.00421633
[4]	train-merror:0.238711+0.00335642	test-merror:0.308342+0.00425407
[5]	train-merror:0.225688+0.00421116	test-merror:0.301+0.0041131
[6]	train-merror:0.21499+0.00321578	test-merror:0.294966+0.00413244
[7]	train-merror:0.205486+0.00334183	test-merror:0.290165+0.00381773
[8]	train-merror:0.196875+0.00386585	test-merror:0.285287+0.00425179
[9]	train-merror:0.188+0.0041378	test-merror:0.281264+0.0029731
[10]	train-merror:0.180507+0.00284265	test-merror:0.278071+0.00284713
[11]	train-merror:0.174021+0.00339272	test-merror:0.275079+0.00341297
[12]	train-merror:0.166428+0.00343022	test-merror:0.271961+0.00353843
[13]	train-merror:0.160582+0.00320031	test-merror:0.270503+0.00332147
[14]	train-merror:0.154661+

[117]	train-merror:0.0212073+0.000799467	test-merror:0.222381+0.00332881
[118]	train-merror:0.020843+0.00080077	test-merror:0.222281+0.00316482
[119]	train-merror:0.0205913+0.000758687	test-merror:0.222256+0.00343563
[120]	train-merror:0.0202897+0.000686015	test-merror:0.222381+0.00351636
[121]	train-merror:0.0199753+0.000558517	test-merror:0.222256+0.00354073
[122]	train-merror:0.0195103+0.000528193	test-merror:0.222206+0.00329875
[123]	train-merror:0.0192087+0.000634595	test-merror:0.222407+0.00327279
[124]	train-merror:0.018957+0.000478601	test-merror:0.222784+0.00348173
[125]	train-merror:0.0186553+0.000479273	test-merror:0.222733+0.0035381
[126]	train-merror:0.018467+0.000479317	test-merror:0.222583+0.00341746
[127]	train-merror:0.0181023+0.000539534	test-merror:0.222231+0.0033866
[128]	train-merror:0.0178383+0.000601124	test-merror:0.222029+0.003451
[129]	train-merror:0.0176243+0.000617074	test-merror:0.22208+0.00370612
[130]	train-merror:0.0172977+0.00050979	test-merror:0.221904

[230]	train-merror:0.004488+0.000263469	test-merror:0.218484+0.00383714
[231]	train-merror:0.00445+0.000246582	test-merror:0.21851+0.00384449
[232]	train-merror:0.00437467+0.000277201	test-merror:0.218384+0.00382284
[233]	train-merror:0.004274+0.000246808	test-merror:0.218258+0.00394928
[234]	train-merror:0.00419867+0.000231242	test-merror:0.218258+0.00404989
[235]	train-merror:0.00414833+0.000184937	test-merror:0.218183+0.00404544
[236]	train-merror:0.004111+0.000215555	test-merror:0.218258+0.00415607
[237]	train-merror:0.004073+0.000184528	test-merror:0.218333+0.00408919
[238]	train-merror:0.004023+0.000218435	test-merror:0.218434+0.00416402
[239]	train-merror:0.00397267+0.000247232	test-merror:0.218535+0.00402956
[240]	train-merror:0.00390967+0.000292844	test-merror:0.21851+0.00410769
[241]	train-merror:0.00385933+0.000295645	test-merror:0.218585+0.00416014
[242]	train-merror:0.00378367+0.000286136	test-merror:0.218535+0.00423154
[243]	train-merror:0.00372067+0.000279476	test-merror

In [26]:
cvresult

,test-merror-mean,test-merror-std,train-merror-mean,train-merror-std
0,0.402801,0.006009,0.369852,0.011453
1,0.348469,0.007151,0.300787,0.006878
2,0.328104,0.003680,0.271848,0.006072
3,0.317846,0.004216,0.254362,0.005663
4,0.308342,0.004254,0.238711,0.003356
5,0.301000,0.004113,0.225688,0.004211
6,0.294966,0.004132,0.214990,0.003216
7,0.290165,0.003818,0.205486,0.003342
8,0.285287,0.004252,0.196875,0.003866
9,0.281264,0.002973,0.188000,0.004138


In [27]:
cvresult.shape[0]

279